In [10]:
import os
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
from circadian.models import Skeldon23
from utils.helper_functions import H_thresholds

In [11]:
save_path = "data/chang14_selected_sleep_parameters/"
if not os.path.exists(save_path):
    os.makedirs(save_path)

In [12]:
forced_wakeup_threshold = 0.5 # lux

light_data_location = 'data/light_schedules/'
time = np.load(f"{light_data_location}/time.npy")
regular_light = np.load(f"{light_data_location}/regular_light.npy")
chang14_ebook_first_light = np.load(f"{light_data_location}/chang14_ebook_first_light.npy")
chang14_ebook_second_light = np.load(f"{light_data_location}/chang14_ebook_second_light.npy")
realistic_ebook_first_light = np.load(f"{light_data_location}/realistic_ebook_first_light.npy")
realistic_ebook_second_light = np.load(f"{light_data_location}/realistic_ebook_second_light.npy")

for idx in tqdm(selected_parameters.index, desc='Simulating', total=len(selected_parameters)):
    # Parameters
    mu = selected_parameters.loc[idx, 'mu']
    delta = selected_parameters.loc[idx, 'Delta']
    chi = selected_parameters.loc[idx, 'chi']
    # Initial condition
    initial_condition = np.array([
        selected_parameters.loc[idx, 'x_f'],
        selected_parameters.loc[idx, 'xc_f'],
        selected_parameters.loc[idx, 'n_f'],
        selected_parameters.loc[idx, 'H_f'],
    ])
    S0 = selected_parameters.loc[idx, 'S_f']
    parameters = {
        'S0': S0,
        'mu': mu, 'Delta': delta, 'chi': chi,
        'forced_wakeup_light_threshold': forced_wakeup_threshold,
    }
    # Regular simulation
    regular_model = Skeldon23(params=parameters)
    regular_trajectory = regular_model.integrate(time, 
                                                 initial_condition=initial_condition,
                                                 input=regular_light)
    regular_H_plus, regular_H_minus = H_thresholds(regular_model, regular_trajectory)
    # Chang14 Ebook first simulation
    chang14_ebook_first_model = Skeldon23(params=parameters)
    chang14_ebook_first_trajectory = chang14_ebook_first_model.integrate(time, 
                                                                         initial_condition=initial_condition, 
                                                                         input=chang14_ebook_first_light)
    chang14_ebook_first_H_plus, chang14_ebook_first_H_minus = H_thresholds(chang14_ebook_first_model, chang14_ebook_first_trajectory)
    # Chang14 Ebook second simulation
    chang14_ebook_second_model = Skeldon23(params=parameters)
    chang14_ebook_second_trajectory = chang14_ebook_second_model.integrate(time, 
                                                          initial_condition=initial_condition,
                                                          input=chang14_ebook_second_light)
    chang14_ebook_second_H_plus, chang14_ebook_second_H_minus = H_thresholds(chang14_ebook_second_model, chang14_ebook_second_trajectory)
    # Realistic Ebook first simulation
    realistic_ebook_first_model = Skeldon23(params=parameters)
    realistic_ebook_first_trajectory = realistic_ebook_first_model.integrate(time, 
                                                                             initial_condition=initial_condition, 
                                                                             input=realistic_ebook_first_light)
    realistic_ebook_first_H_plus, realistic_ebook_first_H_minus = H_thresholds(realistic_ebook_first_model, realistic_ebook_first_trajectory)
    # Realistic Ebook second simulation
    realistic_ebook_second_model = Skeldon23(params=parameters)
    realistic_ebook_second_trajectory = realistic_ebook_second_model.integrate(time, 
                                                                              initial_condition=initial_condition,
                                                                              input=realistic_ebook_second_light)
    realistic_ebook_second_H_plus, realistic_ebook_second_H_minus = H_thresholds(realistic_ebook_second_model, realistic_ebook_second_trajectory)
    # Save data
    np.savez(f"{save_path}/sleep_parameters_chang14_simulation_{idx}.npz",
             time=time,
             parameters=parameters,
             initial_condition=initial_condition,
             regular_trajectory_states=regular_trajectory.states,
             regular_light=regular_light,
             regular_sleep=regular_model.sleep_state,
             regular_H_plus=regular_H_plus,
             regular_H_minus=regular_H_minus,
             chang14_ebook_first_trajectory_states=chang14_ebook_first_trajectory.states,
             chang14_ebook_first_light=chang14_ebook_first_light,
             chang14_ebook_first_sleep=chang14_ebook_first_model.sleep_state,
             chang14_ebook_first_H_plus=chang14_ebook_first_H_plus,
             chang14_ebook_first_H_minus=chang14_ebook_first_H_minus,
             chang14_ebook_second_trajectory_states=chang14_ebook_second_trajectory.states,
             chang14_ebook_second_light=chang14_ebook_second_light,
             chang14_ebook_second_sleep=chang14_ebook_second_model.sleep_state,
             chang14_ebook_second_H_plus=chang14_ebook_second_H_plus,
             chang14_ebook_second_H_minus=chang14_ebook_second_H_minus,
             realistic_ebook_first_trajectory_states=realistic_ebook_first_trajectory.states,
             realistic_ebook_first_light=realistic_ebook_first_light,
             realistic_ebook_first_sleep=realistic_ebook_first_model.sleep_state,
             realistic_ebook_first_H_plus=realistic_ebook_first_H_plus,
             realistic_ebook_first_H_minus=realistic_ebook_first_H_minus,
             realistic_ebook_second_trajectory_states=realistic_ebook_second_trajectory.states,
             realistic_ebook_second_light=realistic_ebook_second_light,
             realistic_ebook_second_sleep=realistic_ebook_second_model.sleep_state,
             realistic_ebook_second_H_plus=realistic_ebook_second_H_plus,
             realistic_ebook_second_H_minus=realistic_ebook_second_H_minus,
    )

Simulating:   0%|          | 0/61 [00:00<?, ?it/s]